In [1]:
### This notebook select empty window and safe harbor sgRNAs and find the optimal primers to the sgRNA sequences
import pandas as pd
import sys
sys.path.append('../')
from src.utils import *
import random

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
out_path = '../../data/sg_Finder'

ABE_synsg = org_sgdf(join(out_path,'ess_15','ABE_synsg.csv'))
CBE_synsg = org_sgdf(join(out_path,'ess_15','CBE_synsg.csv'))
neutral_ABE_synsg = org_sgdf(join(out_path,'ess_15','ABE_synsg.csv'))
neutral_CBE_synsg = org_sgdf(join(out_path,'ess_15','CBE_synsg.csv'))

In [6]:
print(f'There are {len(ABE_synsg)} ABE sgRNA and {len(CBE_synsg)} CBE sgRNA')

There are 2134 ABE sgRNA and 3528 CBE sgRNA


In [7]:
ABE_etywind = org_sgdf(join(out_path,'ess_15','ABE_CT.csv'))
CBE_etywind = org_sgdf(join(out_path,'ess_15','CBE_CT.csv'))
# Select empty window sg for at least one sg per gene
sel_ABE_etywind = sel_empty_window(ABE_etywind)
sel_CBE_etywind = sel_empty_window(CBE_etywind)

NameError: name 'sel_empty_window' is not defined

In [ ]:
print(f'There are {len(sel_ABE_etywind)} control ABE sg and {len(sel_CBE_etywind)} control CBE sg')

In [11]:
print(f'There are {len(sel_ABE_etywind)} control ABE sg and {len(sel_CBE_etywind)} control CBE sg')

There are 238 control ABE sg and 236 control CBE sg


In [7]:
df_AAVS1 = readnCT('Final_sgAAVS1.txt',sel = True)
df_CCR5 = readnCT('Final_sgCCR5.txt',sel = True)
df_NT = readnCT('Final_sgNT.txt',sel = True)

In [22]:
len(df_AAVS1)+len(df_CCR5)+len(df_NT)

120

In [9]:
### Combine negative control and sample together!
df_abe = pd.concat([ABE_synsg, sel_ABE_etywind, df_AAVS1,df_CCR5,df_NT], axis = 0).reset_index()
df_cbe = pd.concat([CBE_synsg, sel_CBE_etywind, df_AAVS1,df_CCR5,df_NT], axis = 0).reset_index()

In [23]:
len(df_abe)

3941

In [24]:
len(df_cbe)

6785

In [12]:
### Positive control set: sg from screen result in enrich/deplete fitness
sglabel = pd.read_csv(join(ivan_path,'BCRABBL.csv'))
labels = sglabel[~sglabel[['sgNAME', 'sgRNA Seq']].duplicated()][['sgNAME', 'sgRNA Seq']].set_index('sgNAME')
list_posCT_ABE = getpCT('K562_CD_Fitness.gene_summary.txt', [5,30], labels)
list_posCT_CBE = getpCT('K562_AB_Fitness.gene_summary.txt', [1,25], labels)
df_pCT_ABE = make_sg_df(list_posCT_ABE,'BCRABL')
df_pCT_CBE = make_sg_df(list_posCT_CBE,'BCRABL')

In [16]:
len(df_pCT_ABE)

50

In [17]:
len(df_pCT_CBE)

45

In [ ]:
# ABE : negative, posCTABE, sample set
# CBE: negative, posCT CBE, sample set
# Four primers : one for negative+sample ABE, negative+sample CBE, pos ABE, posCBE

In [14]:
list_abe = list(df_abe.sgRNA);list_cbe = list(df_cbe.sgRNA)
rABE_noprimer_list,rABE_pass_list = get_noprimer_list(list_abe)
rCBE_noprimer_list,rCBE_pass_list = get_noprimer_list(list_cbe)
print(f'{len(rABE_noprimer_list)} ABE sg groups and {len(rCBE_noprimer_list)} CBE sg groups')

1940 ABE sg groups and 3373 CBE sg groups


In [15]:
rABE_noprimer_list_pos,rABEpos_pass_list = get_noprimer_list(list_posCT_ABE)
rCBE_noprimer_list_pos,rCBEpos_pass_list = get_noprimer_list(list_posCT_CBE)
print(f'{len(rABE_noprimer_list_pos)} ABE positive control\
sg groups and {len(rCBE_noprimer_list_pos)} CBE positive control sg groups')

25 ABE positive controlsg groups and 21 CBE positive control sg groups


In [114]:
#Save Fasta for cross refrences agaisnt potential primers
write_noprime_list('rREsg_ABE.fasta',rABE_noprimer_list)
write_noprime_list('rREsg_CBE.fasta',rCBE_noprimer_list)
write_noprime_list('rREsg_CBE_pCT.fasta',rCBE_noprimer_list_pos)
write_noprime_list('rREsg_ABE_pCT.fasta',rABE_noprimer_list_pos)

***Get some statistics about gene and their sgRNAs***

In [ ]:
# Load all expressed transcripts
f_genelist = pd.read_csv('../../data/output/k562_comm_ess.txt', header =None)
list_exp_trsp = pickle.load(open('../../data/22Q2_gene_effect/expressed_transcripts.pkl','rb'))
# Parse mRNA sequence dictionary, for use when sequence aligned to intron-exon junctions
record_dict = SeqIO.to_dict(SeqIO.parse(join(mrna_path,'GRCh38_latest_rna.fna'), "fasta"))

***Blast n all possible primers***

In [ ]:
# cd /data/output/primers
# makeblastdb -in rREsg_ABE.fasta -dbtype nucl -parse_seqids -max_file_sz 4000000000
# blastn -query PossiblePrimers.fasta -db rREsg_ABE.fasta -out Blast_Primer_ABE.txt -word_size 7 -dust no

# makeblastdb -in rREsg_CBE.fasta -dbtype nucl -parse_seqids -max_file_sz 4000000000
# blastn -query PossiblePrimers.fasta -db rREsg_CBE.fasta -out Blast_Primer_CBE.txt -word_size 7 -dust no

#or
# blastn -query PossiblePrimers.fasta -db rREsg_CBE.fasta -out Blast_Primer_CBE.txt -word_size 7 -dust no -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore staxids"
#https://www.biostars.org/p/141781/


In [ ]:
BlastQueryPrimer_ABE = read_blast_res('Blast_Primer_ABE.txt')
BlastQueryPrimer_CBE = read_blast_res('Blast_Primer_CBE.txt')

In [ ]:
PrimerHeteroTM_ABE = calc_TM(BlastQueryPrimer_ABE[0:50],rABE_noprimer_list)
PrimerHeteroTM_CBE = calc_TM(BlastQueryPrimer_CBE[0:50],rCBE_noprimer_list)

In [ ]:
GoodPrimer_ABE = sel_primers(22,PrimerHeteroTM_ABE)
GoodPrimer_CBE = sel_primers(22,PrimerHeteroTM_CBE)

In [ ]:
Fwd,Rev,oligoTM, primerTM = calc_best_primerTM(GoodPrimer_CBE, rCBE_noprimer_list)

In [ ]:
CBE_final_seq = attach_group_primer(rABE_pass_list,Fwd[:18],Rev[:18])

***Design Primer***  
Actually should be a seperate part  
Doesn't need to run this part!

In [36]:
random.seed(1)

In [63]:
# Theoretically, a length of 20, # allele =4 has 4^20 possible sequences, 200000 is < 4^9
# But that's already a lot
# From Ivan:
PossiblePrimer = [] 
while len(PossiblePrimer) < 200000:
    primer = random.choice(['C','G']) + ''.join(random.choice(['A','G','C','T']) for i in range(17)) + 'CG'
    #Check if it contains cut sites or repeat
    if not any(bad in primer for bad in BadSites):
        #Balanced GC Content
        if (primer.count('C') + primer.count('G')) == 10:
            #GC Clamp 
            if 4 > (primer.count('C', 15, 20) + primer.count('G', 15, 20)):
                #Hairpin
                if p3.calcHairpin(primer).tm < 10:
                    #Homodimer
                    if p3.calcHomodimer(primer).tm < 10:
                        PossiblePrimer.append(primer)
    #remove duplicates
PossiblePrimer = list(set(PossiblePrimer))

In [66]:
len(PossiblePrimer) # This actually means the search is saturated

199982

In [65]:
with open('../../data/output/primers/PossiblePrimers.fasta', 'w') as f:
    for i in PossiblePrimer:
        f.write('>'+i)
        f.write('\n')
        f.write(i)
        f.write('\n')